In [172]:
# Model 분석

In [173]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

df1 = pd.read_csv('./data/df.csv')

In [174]:
# id, 이름, 시설종류, 평점, 투표횟수, 
# 실내/실외, 종목
df_main = df1[ ['ID','이름', '대분류','소분류', '평점', '투표횟수','실내/실외', '태깅'] ]

In [175]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환. 
# df_main['종목_정확도'] = df_main['종목'].apply(lambda x : (' ').join( x ))
count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
genre_mat = count_vect.fit_transform( df_main['태깅'] )
genre_mat.shape

(203, 511)

In [191]:
# print( genre_mat[:1] )
# genre_mat[:1].toarray()

In [177]:
# 코사인 유사도 계산계산
# 반환된 코사인 유사도 행렬의 크기 및 앞 2개 데이터만 추출
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)

In [178]:
# 유사도가 높은 순으로 정리된 genre_sim 객체의 비교 행 위치 인덱스 값
# 값이 높은 순으로 정렬된 비교 대상 행의 유사도 값이 아니라
# 비교 대상 행의 위치 인덱스임에 주의
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]

In [179]:
C = df_main['평점'].mean()
m = df_main['투표횟수'].quantile(0.6)

In [180]:
# 기존 평점을 가중 평점으로 변경하는 함수
def weighted_vote_average(record):
  v = record['투표횟수']
  R = record['평점']
  # (예정)날씨 관련 수식을 추가 => 실내외 구분시 활용
  return ( ( (v/(v+m)) * R ) + ( (m/(v+m)) * C ) ) *2

df_main['추천점수'] = df_main.apply(weighted_vote_average, axis=1)

In [181]:
df_temp = df_main[['이름','대분류', '소분류', '실내/실외', '평점', '투표횟수', '추천점수']]
df_temp[df_main['소분류']=='미술관']


,이름,대분류,소분류,실내/실외,평점,투표횟수,추천점수
0,드영미술관,예술,미술관,실내,3.0,1,7.466283
1,이이남 스튜디오,예술,미술관,실내,4.7,84,8.201914
2,광주시립미술관,예술,미술관,실내,4.3,815,8.436555
3,소촌아트팩토리,예술,미술관,실내,4.1,53,7.676676
4,광주여성전시관 Herstory,예술,미술관,실내,0.0,0,7.476847
5,국윤미술관,예술,미술관,실내,3.9,7,7.492362
6,롯데갤러리 광주점,예술,미술관,실내,0.0,0,7.476847
7,무등갤러리,예술,미술관,실내,4.3,11,7.559322
8,무등현대미술관,예술,미술관,실내,3.9,55,7.568557
9,북구청갤러리,예술,미술관,실내,0.0,0,7.476847


In [182]:
# 새롭게 부여된 가중치 평점이 높은 순으로 상위 10개의 시설
df_head_1 = df_main[['이름','대분류', '소분류', '실내/실외', '평점', '투표횟수', '추천점수']][df_main['소분류']=='미술관'].sort_values(
    '추천점수', ascending=False).head(1)
df_head_2 = df_main[['이름','대분류', '소분류', '실내/실외', '평점', '투표횟수', '추천점수']][df_main['소분류']=='박물관'].sort_values(
    '추천점수', ascending=False).head(1)
df_head_3 = df_main[['이름','대분류', '소분류', '실내/실외', '평점', '투표횟수', '추천점수']][df_main['소분류']=='공연장'].sort_values(
    '추천점수', ascending=False).head(1)
df_head_4 = df_main[['이름','대분류', '소분류', '실내/실외', '평점', '투표횟수', '추천점수']][df_main['소분류']=='전통문화'].sort_values(
    '추천점수', ascending=False).head(1)
df_head_5 = df_main[['이름','대분류', '소분류', '실내/실외', '평점', '투표횟수', '추천점수']][df_main['소분류']=='5.18운동'].sort_values(
    '추천점수', ascending=False).head(1)
df_head_6 = df_main[['이름','대분류', '소분류', '실내/실외', '평점', '투표횟수', '추천점수']][df_main['소분류']=='사찰/기독교'].sort_values(
    '추천점수', ascending=False).head(1)
df_head_7 = df_main[['이름','대분류', '소분류', '실내/실외', '평점', '투표횟수', '추천점수']][df_main['소분류']=='선비문화'].sort_values(
    '추천점수', ascending=False).head(1)
df_head_8 = df_main[['이름','대분류', '소분류', '실내/실외', '평점', '투표횟수', '추천점수']][df_main['소분류']=='문화유산'].sort_values(
    '추천점수', ascending=False).head(1)
df_head_9 = df_main[['이름','대분류', '소분류', '실내/실외', '평점', '투표횟수', '추천점수']][df_main['소분류']=='체험'].sort_values(
    '추천점수', ascending=False).head(1)
df_head_10 = df_main[['이름','대분류', '소분류', '실내/실외', '평점', '투표횟수', '추천점수']][df_main['소분류']=='공원'].sort_values(
    '추천점수', ascending=False).head(1)
df_head_11 = df_main[['이름','대분류', '소분류', '실내/실외', '평점', '투표횟수', '추천점수']][df_main['소분류']=='맛집'].sort_values(
    '추천점수', ascending=False).head(1)
df_head_12 = df_main[['이름','대분류', '소분류', '실내/실외', '평점', '투표횟수', '추천점수']][df_main['소분류']=='시장'].sort_values(
    '추천점수', ascending=False).head(1)
df_head_13 = df_main[['이름','대분류', '소분류', '실내/실외', '평점', '투표횟수', '추천점수']][df_main['소분류']=='스포츠'].sort_values(
    '추천점수', ascending=False).head(1)
df_head_14 = df_main[['이름','대분류', '소분류', '실내/실외', '평점', '투표횟수', '추천점수']][df_main['소분류']=='마을'].sort_values(
    '추천점수', ascending=False).head(1)

In [183]:
result_df = pd.concat([df_head_1,df_head_2,df_head_3,df_head_4,df_head_5,df_head_6,df_head_7,df_head_8,df_head_9,df_head_10,df_head_11,df_head_12,df_head_13, df_head_14])
result_df.sort_values('추천점수', ascending=False)

,이름,대분류,소분류,실내/실외,평점,투표횟수,추천점수
52,5·18민주화운동기록관,역사,5.18운동,실내,4.6,636,8.891309
184,쌍암공원,관광,공원,실외,4.5,1119,8.831918
99,국립광주과학관,축제/체험,체험,실내,4.4,1644,8.696986
138,100족발본점,먹거리,맛집,실내,4.8,187,8.695477
32,국립아시아문화전당,예술,공연장,실내,4.6,279,8.627540
25,국립광주박물관,예술,박물관,실내,4.3,955,8.457475
2,광주시립미술관,예술,미술관,실내,4.3,815,8.436555
65,무각사,역사,사찰/기독교,실내,4.4,354,8.427326
182,광주월드컵경기장,관광,스포츠,실외,4.3,597,8.388130
78,포충사,역사,선비문화,실외,4.4,268,8.348541


In [184]:
# 날씨기반 실내/외 판별 함수
# a = 온도 , b = 습도 , c = 1시간당 강수량

def weather_choice(a,b,c):
  y = b - ((-4.3 * a) + 147)
  if y >= 10:
    a = a + (y/10)

  if c >= 5 or a<=10 or a>=30:
    return '실내'
  else:
    return '실외'

In [185]:
# 새롭게 정의된 평점 기준에 따라 기존 find_sim_experience 함수를 변경
def find_sim_experience(df_main, sorted_ind, title_name, top_n=10, end_idx=10):
  title_exp = df_main[df_main['이름'] == title_name]
  title_index = title_exp.index.values

  sim_index = sorted_ind[title_index, :top_n*3]
  sim_index = sim_index.reshape(-1)

  sim_index = sim_index[sim_index != title_index]

  return df_main.iloc[sim_index].sort_values(
    '추천점수', ascending=False)[:end_idx]

def find_sim_experience_rainy(df_main, sorted_ind, title_name, top_n=10):
  title_exp = df_main[df_main['이름'] == title_name]
  title_index = title_exp.index.values

  sim_index = sorted_ind[title_index, :top_n*3]
  sim_index = sim_index.reshape(-1)

  sim_index = sim_index[sim_index != title_index]

  return df_main.iloc[sim_index].sort_values(
    '추천점수', ascending=False)[:]

input_name = '증심사'
sim_exp = find_sim_experience(df_main, genre_sim_sorted_ind, input_name)
sim_exp_rainy = find_sim_experience_rainy(df_main, genre_sim_sorted_ind, input_name)
test = sim_exp[['이름', '대분류', '소분류', '실내/실외', '평점', '투표횟수', '추천점수']]
test_rainy = sim_exp_rainy[['이름', '대분류', '소분류', '실내/실외', '평점', '투표횟수', '추천점수']]

In [186]:
import pandas as pd

df2 = pd.read_csv("./data/weather.csv")

In [187]:
# # 기상청 API에서 온도, 습도, 강수량 가져오기

# import requests
# from datetime import datetime, timedelta
# from pprint import pprint


# url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst"
# service_key = "OsnExmgat6fBnPcmy5/u/Ish125BWuL2gaRjXIUpbldSNUyz6obA1tlae77HEiPh67zsMKlW3nJ5MS4G/6G7jA=="

# now = datetime.now()

# if now.minute <= 40:
#     if now.hour == 0 :
#         date = (now-timedelta(days=1)).strftime('%Y%m%d')
#         date = '2300'
#     else:
#         date = now.strftime('%Y%m%d')
#         time = (now-timedelta(hours=1)).strftime('%H00')
# else:
#     date = base_date = now.strftime('%Y%m%d') 
#     time = now.strftime('%H00')

# params = {
#     'serviceKey' : service_key,
#     'numOfRows' : 50,
#     'pageNo' : 1,
#     'dataType' : 'JSON',
#     'base_date' : date,
#     'base_time' : time,
#     'nx' : 59,
#     'ny' : 74 
# }

# res = requests.get(url=url , params=params)

# data = res.json()
# data = data['response']['body']['items']['item']
# pprint(data)

In [188]:
# 날씨로 실내/실외 필터링
i = 0 # 맑음 : 0 / 비 : 1

temper = df2.loc[i][0:][0] # 온도
humid = df2.loc[i][0:][1]  # 습도
rain = df2.loc[i][0:][2]   # 강수량
    
    # 실내만 추천
if weather_choice(temper, humid, rain) == "실내":
    print(test_rainy[test_rainy['실내/실외'] == "실내"][:10])
    
    # 실내/외 모두 추천
else :
    print(test)

              이름 대분류     소분류 실내/실외   평점  투표횟수      추천점수
52  5·18민주화운동기록관  역사  5.18운동    실내  4.6   636  8.891309
55      5·18민주광장  역사  5.18운동    실외  4.6   488  8.818421
53       전일빌딩245  역사  5.18운동    실내  4.7   240  8.695318
56      5·18자유공원  역사  5.18운동    실외  4.5   360  8.576156
54      5·18기념공원  역사  5.18운동    실외  4.4   574  8.542349
65           무각사  역사  사찰/기독교    실내  4.4   354  8.427326
57   광주학생독립운동기념관  역사  5.18운동    실내  4.6   143  8.351265
78           포충사  역사    선비문화    실외  4.4   268  8.348541
69           원효사  역사  사찰/기독교    실내  4.3   411  8.316454
63     우일선 선교사사택  역사  사찰/기독교    실내  4.4   166  8.197462
